## Set the repository path

In [ ]:
pyCPT_path = '../'

## Import modules

In [ ]:
# Add the repository path into python path
import sys
sys.path.append(pyCPT_path)

# Import modules
import pyCPT
import post_process

## Read the NGES dataset and identify the optimal number of soil states
#### Note: 1) Data file should contain 3 columns: ( depth, Fr, Qt ); 2) Data should start from the 1st row!

In [ ]:
datapath = '../test_data/NGES_data.csv'
cpt = pyCPT.CPT(datapath)

## Extract soil segments

In [ ]:
cpt.segmentation(num_of_iter=200, start_iter=100)

## Soil interpretation

In [ ]:
cpt.detect_layers()

## Visualization

In [ ]:
post_process.plot_feature(cpt.element,xlabel=['Fr','Qt'], scatter=False) # show the CPT data
post_process.plot_image(cpt.element, cpt.element.label_map_est) # show the spatial pattern
pyCPT.mixture_plot_robertson_chart(cpt, aspect=0.8) # show the statistical pattern
pyCPT.plot_layers(cpt) # show the soil layers

#### Identified issue: When you test the repository by using the on-line interactive binder version, some noise contour lines could show up in the contour plot on the Roberson chart. This is an identified issue and probabily related to the version of the package "matplotlib" in MyBinder.org. The colone version from the github repository does not have such behavior.